In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import src.config as config

In [84]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1049751
Connected. Call `.close()` to terminate connection gracefully.


In [85]:
# create feature view (if it doesn't exist yet)
# This feature view only uses one feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


Create training data from the feature view data.

In [86]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (29.56s) 
2024-10-08 17:07:47,896 WARNING: c:\Users\ryans\taxi_demand_predictor\.venv\lib\site-packages\hsfs\feature_view.py:2231: VersionWarning:

Incremented version to `4`.




In [87]:
ts_data

,pickup_hour,rides,pickup_location_id,pickup_ts
0,2023-01-05 14:00:00+00:00,3,65,1672927200000
1,2022-02-05 08:00:00+00:00,0,136,1644048000000
2,2023-10-08 04:00:00+00:00,13,162,1696737600000
3,2022-10-23 18:00:00+00:00,0,98,1666548000000
4,2023-08-13 18:00:00+00:00,0,180,1691949600000
...,...,...,...,...
5000790,2022-07-04 13:00:00+00:00,1,102,1656939600000
5000791,2022-11-15 13:00:00+00:00,4,116,1668517200000
5000792,2022-03-05 05:00:00+00:00,0,40,1646456400000
5000793,2024-06-27 20:00:00+00:00,51,151,1719518400000


In [88]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
2775676,2022-01-01 00:00:00+00:00,0,1
1353163,2022-01-01 01:00:00+00:00,0,1
3603306,2022-01-01 02:00:00+00:00,0,1
3932308,2022-01-01 03:00:00+00:00,0,1
4718819,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
1775226,2024-10-08 18:00:00+00:00,7,265
1881025,2024-10-08 19:00:00+00:00,4,265
1929748,2024-10-08 20:00:00+00:00,2,265
1930030,2024-10-08 21:00:00+00:00,3,265


In [89]:
ts_data

,pickup_hour,rides,pickup_location_id
2775676,2022-01-01 00:00:00+00:00,0,1
1353163,2022-01-01 01:00:00+00:00,0,1
3603306,2022-01-01 02:00:00+00:00,0,1
3932308,2022-01-01 03:00:00+00:00,0,1
4718819,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
1775226,2024-10-08 18:00:00+00:00,7,265
1881025,2024-10-08 19:00:00+00:00,4,265
1929748,2024-10-08 20:00:00+00:00,2,265
1930030,2024-10-08 21:00:00+00:00,3,265


In [91]:
len(ts_data.loc[ts_data['pickup_location_id'] == 265])

675

In [92]:
ts_data['pickup_hour'].info()

<class 'pandas.core.series.Series'>
Int64Index: 5000795 entries, 2775676 to 1930432
Series name: pickup_hour
Non-Null Count    Dtype 
--------------    ----- 
5000795 non-null  object
dtypes: object(1)
memory usage: 76.3+ MB


In [93]:
from src.plot import plot_ts

plot_ts(ts_data, locations=[265])

In [95]:
import pandas as pd
# Convert pickup_hour to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

In [96]:
ts_data

,pickup_hour,rides,pickup_location_id
2775676,2022-01-01 00:00:00+00:00,0,1
1353163,2022-01-01 01:00:00+00:00,0,1
3603306,2022-01-01 02:00:00+00:00,0,1
3932308,2022-01-01 03:00:00+00:00,0,1
4718819,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
1775226,2024-10-08 18:00:00+00:00,7,265
1881025,2024-10-08 19:00:00+00:00,4,265
1929748,2024-10-08 20:00:00+00:00,2,265
1930030,2024-10-08 21:00:00+00:00,3,265


In [97]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data, 
    input_seq_len = 24*28*1,   # one month of features
    step_size = 23
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

100%|██████████| 265/265 [01:00<00:00,  4.40it/s]

features.shape=(209913, 674)
targets.shape=(209913,)


In [98]:
features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

In [99]:
print(f'{features_and_target.shape=}')

features_and_target.shape=(209913, 675)


In [100]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29 00:00:00+00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2022-01-29 23:00:00+00:00,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,2.0,2.0,0.0,1.0,2.0,0.0,0.0,2022-01-30 22:00:00+00:00,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,1.0,0.0,1.0,2022-01-31 21:00:00+00:00,1,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-02-01 20:00:00+00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209908,58.0,37.0,27.0,11.0,6.0,7.0,4.0,1.0,3.0,3.0,...,37.0,46.0,41.0,61.0,71.0,54.0,40.0,2024-10-08 20:00:00+00:00,261,35.0
209909,78.0,28.0,19.0,6.0,2.0,2.0,1.0,3.0,7.0,20.0,...,100.0,73.0,122.0,83.0,99.0,97.0,82.0,2024-10-08 20:00:00+00:00,262,39.0
209910,111.0,90.0,67.0,36.0,24.0,9.0,7.0,1.0,6.0,15.0,...,100.0,100.0,120.0,126.0,112.0,112.0,128.0,2024-10-08 20:00:00+00:00,263,92.0
209911,62.0,56.0,51.0,30.0,16.0,9.0,6.0,2.0,7.0,11.0,...,31.0,52.0,46.0,57.0,68.0,41.0,44.0,2024-10-08 20:00:00+00:00,264,51.0


In [12]:
features_and_target['pickup_hour'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 209913 entries, 0 to 209912
Series name: pickup_hour
Non-Null Count   Dtype 
--------------   ----- 
209913 non-null  object
dtypes: object(1)
memory usage: 1.6+ MB


In [13]:
import pandas as pd
# Convert pickup_hour to datetime
features_and_target['pickup_hour'] = pd.to_datetime(features_and_target['pickup_hour'], utc=True)

In [14]:
len(features_and_target.pickup_location_id.unique())

265

In [29]:
len(features_and_target)

209913

In [41]:
locations = features_and_target['pickup_location_id'].unique()

In [42]:
locations

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [48]:
985 * 265

261025

In [47]:
data = []
for i in locations:

    data.append(len(features_and_target.loc[features_and_target['pickup_location_id'] == i]))
print(data)

[985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985, 985,

In [20]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 1 month ago
# test data -> last 2 months
# Convert pickup_hour to datetime
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-09-10 00:00:00+0000', tz='UTC')
X_train.shape=(203443, 674)
y_train.shape=(203443,)
X_test.shape=(6470, 674)
y_test.shape=(6470,)


In [32]:
203443 + 6470

209913

In [21]:
X_test['pickup_location_id'].max()

265

In [22]:
X_train['pickup_location_id'].max()

214

In [23]:
X_test['pickup_location_id'].min()

1

In [28]:
X_test.loc[X_test['pickup_location_id'] == 1]

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,...,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2024-09-11 01:00:00+00:00,1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2024-09-12 00:00:00+00:00,1
2,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,0.0,...,0.0,2.0,2.0,1.0,2.0,2.0,1.0,0.0,2024-09-12 23:00:00+00:00,1
3,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,1.0,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-09-13 22:00:00+00:00,1
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2024-09-14 21:00:00+00:00,1
5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,...,0.0,3.0,3.0,1.0,2.0,1.0,0.0,1.0,2024-09-15 20:00:00+00:00,1
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,2024-09-16 19:00:00+00:00,1
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,2024-09-17 18:00:00+00:00,1
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,1.0,0.0,2.0,2024-09-18 17:00:00+00:00,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,2024-09-19 16:00:00+00:00,1
